### Load the Required Libraries

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._


#### Create SparkSession

In [3]:
val spark = SparkSession.
            builder().
            appName("MovieLens").
            config("spark.sql.warehouse.dir", "tmp/sparksql").
            getOrCreate()
            

#### Create Schema for Ratings Data and User Data

In [4]:
val schemaRating = StructType(List(StructField("userId", IntegerType, true),
                                    StructField("movieId",IntegerType, true),
                                    StructField("rating", IntegerType, true), 
                                    StructField("timeStamp", StringType, true)
                                    )
                                 ) 

In [5]:
val schemaUser = StructType(List(StructField("userId", IntegerType, true),
                                    StructField("age", IntegerType, true),
                                    StructField("gender", StringType, true), 
                                    StructField("occupation", StringType, true),
                                    StructField("zipCode", StringType, true)
                                    )
                                )

#### Explore the Data

In [6]:
//Load the Ratings data

val dfRating = spark.read.
                        option("header", false).
                        schema(schemaRating).
                        option("delimiter", "\t").
                        csv("data/u.data")
                            

In [7]:
//Load the Users data

val dfUser = spark.read.
                        option("header", false).
                        schema(schemaUser).
                        option("delimiter", "|").
                        csv("data/u.user")
                            

#### Check the Contents of the loaded data

In [8]:
dfRating.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timeStamp|
+------+-------+------+---------+
|   196|    242|     3|881250949|
|   186|    302|     3|891717742|
|    22|    377|     1|878887116|
|   244|     51|     2|880606923|
|   166|    346|     1|886397596|
+------+-------+------+---------+
only showing top 5 rows



In [9]:
dfUser.show(5)

+------+---+------+----------+-------+
|userId|age|gender|occupation|zipCode|
+------+---+------+----------+-------+
|     1| 24|     M|technician|  85711|
|     2| 53|     F|     other|  94043|
|     3| 23|     M|    writer|  32067|
|     4| 24|     M|technician|  43537|
|     5| 33|     F|     other|  15213|
+------+---+------+----------+-------+
only showing top 5 rows



#### Print Schema

In [10]:
dfRating.printSchema()


root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timeStamp: string (nullable = true)



In [11]:
dfUser.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zipCode: string (nullable = true)



#### Register the DataFrame as a SQL temporary view

In [12]:
dfRating.createOrReplaceTempView("mRating")
dfUser.createOrReplaceTempView("MovieGoer")

#### Select Query for Movie Ratings

In [13]:
//Describe mRating Table
spark.sql("describe mRating").show()

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|   userId|      int|   null|
|  movieId|      int|   null|
|   rating|      int|   null|
|timeStamp|   string|   null|
+---------+---------+-------+



In [14]:
//Total number of ratings
spark.sql("SELECT COUNT(*) FROM mRating").show()

+--------+
|count(1)|
+--------+
|  100000|
+--------+



In [15]:
//Show UserID and the number movies rated
spark.sql("SELECT userId, COUNT(movieId) AS NumOfRatedMovies FROM mRating GROUP BY userId ORDER BY NumOfRatedMovies DESC LIMIT 20").show()

+------+----------------+
|userId|NumOfRatedMovies|
+------+----------------+
|   405|             737|
|   655|             685|
|    13|             636|
|   450|             540|
|   276|             518|
|   416|             493|
|   537|             490|
|   303|             484|
|   234|             480|
|   393|             448|
|   181|             435|
|   279|             434|
|   429|             414|
|   846|             405|
|     7|             403|
|    94|             400|
|   682|             399|
|   308|             397|
|   293|             388|
|    92|             388|
+------+----------------+



In [16]:
//The Number of times the movie has been reviewed and rated
spark.sql("SELECT movieId, count(userId) AS NumOfRatings FROM mRating GROUP BY movieId ORDER BY NumOfRatings DESC LIMIT 20").show()

+-------+------------+
|movieId|NumOfRatings|
+-------+------------+
|     50|         583|
|    258|         509|
|    100|         508|
|    181|         507|
|    294|         485|
|    286|         481|
|    288|         478|
|      1|         452|
|    300|         431|
|    121|         429|
|    174|         420|
|    127|         413|
|     56|         394|
|      7|         392|
|     98|         390|
|    237|         384|
|    117|         378|
|    172|         367|
|    222|         365|
|    313|         350|
+-------+------------+



#### Select Query for User Table

In [17]:
spark.sql("DESCRIBE MovieGoer").show()

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|    userId|      int|   null|
|       age|      int|   null|
|    gender|   string|   null|
|occupation|   string|   null|
|   zipCode|   string|   null|
+----------+---------+-------+



In [18]:
spark.sql("SELECT * FROM MovieGoer").show()

+------+---+------+-------------+-------+
|userId|age|gender|   occupation|zipCode|
+------+---+------+-------------+-------+
|     1| 24|     M|   technician|  85711|
|     2| 53|     F|        other|  94043|
|     3| 23|     M|       writer|  32067|
|     4| 24|     M|   technician|  43537|
|     5| 33|     F|        other|  15213|
|     6| 42|     M|    executive|  98101|
|     7| 57|     M|administrator|  91344|
|     8| 36|     M|administrator|  05201|
|     9| 29|     M|      student|  01002|
|    10| 53|     M|       lawyer|  90703|
|    11| 39|     F|        other|  30329|
|    12| 28|     F|        other|  06405|
|    13| 47|     M|     educator|  29206|
|    14| 45|     M|    scientist|  55106|
|    15| 49|     F|     educator|  97301|
|    16| 21|     M|entertainment|  10309|
|    17| 30|     M|   programmer|  06355|
|    18| 35|     F|        other|  37212|
|    19| 40|     M|    librarian|  02138|
|    20| 42|     F|    homemaker|  95660|
+------+---+------+-------------+-

In [19]:
spark.sql("SELECT COUNT(*)as NUM_Reviewer FROM MovieGoer").show()

+------------+
|NUM_Reviewer|
+------------+
|         943|
+------------+



In [20]:
spark.sql("SELECT gender, COUNT(*) as No FROM MovieGoer GROUP BY gender").show()

+------+---+
|gender| No|
+------+---+
|     F|273|
|     M|670|
+------+---+



In [21]:
spark.sql("SELECT usr.userId, usr.age, usr.gender, usr.occupation, mov.movieId, mov.rating FROM MovieGoer usr JOIN mRating mov ON usr.userId = mov.userId").show()

+------+---+------+----------+-------+------+
|userId|age|gender|occupation|movieId|rating|
+------+---+------+----------+-------+------+
|     1| 24|     M|technician|     94|     2|
|     1| 24|     M|technician|    152|     5|
|     1| 24|     M|technician|    122|     3|
|     1| 24|     M|technician|    172|     5|
|     1| 24|     M|technician|     28|     4|
|     1| 24|     M|technician|     13|     5|
|     1| 24|     M|technician|     88|     4|
|     1| 24|     M|technician|     52|     4|
|     1| 24|     M|technician|    111|     5|
|     1| 24|     M|technician|     15|     5|
|     1| 24|     M|technician|     59|     5|
|     1| 24|     M|technician|     18|     4|
|     1| 24|     M|technician|     29|     1|
|     1| 24|     M|technician|     19|     5|
|     1| 24|     M|technician|    244|     2|
|     1| 24|     M|technician|    207|     5|
|     1| 24|     M|technician|      3|     4|
|     1| 24|     M|technician|    204|     5|
|     1| 24|     M|technician|    